# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [109]:
import psycopg2
import pandas as pd
from pathlib import Path
from helpers import *

In [110]:
# autoreload modules: https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
# https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tree/master/src/jupyter_contrib_nbextensions/nbextensions/ruler
from IPython.html.services.config import ConfigManager
ip = get_ipython()
cm = ConfigManager(parent=ip)
cm.update('notebook', {"ruler_column": [80]})

{'load_extensions': {'jupyng-dist/extension': True,
  'workspaces-jupyter-terms-and-conditions-dist/terms_extension': True,
  'graffiti-dist/graffiti': True},
 'ruler_column': [80]}

In [112]:
!python create_tables.py

In [113]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()
except psycopg2.Error as e:
    print(e)

In [114]:
conn.set_session(autocommit=True)

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [115]:
root_path = Path("/home/workspace")
data_path = root_path.joinpath("data")
song_files = data_path.joinpath("song_data")
log_files = data_path.joinpath("log_data")

In [116]:
song_files = get_files(data_path.joinpath(song_files))

In [117]:
song_files[:5]

['/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEI128F424C983.json',
 '/home/workspace/data/song_data/A/B/C/TRABCKL128F423A778.json',
 '/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEC128F426456E.json']

In [118]:
first_song = song_files[0]

In [119]:
!head $first_song

{"num_songs": 1, "artist_id": "ARNF6401187FB57032", "artist_latitude": 40.79086, "artist_longitude": -73.96644, "artist_location": "New York, NY [Manhattan]", "artist_name": "Sophie B. Hawkins", "song_id": "SONWXQJ12A8C134D94", "title": "The Ballad Of Sleeping Beauty", "duration": 305.162, "year": 1994}

In [120]:
first_df = pd.read_json(first_song, orient="records", typ="series"); first_df

num_songs                                       1
artist_id                      ARNF6401187FB57032
artist_latitude                           40.7909
artist_longitude                         -73.9664
artist_location          New York, NY [Manhattan]
artist_name                     Sophie B. Hawkins
song_id                        SONWXQJ12A8C134D94
title               The Ballad Of Sleeping Beauty
duration                                  305.162
year                                         1994
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [121]:
song_data = first_df[["song_id", "title", "artist_id", "year", "duration"]].values.tolist(); song_data

['SONWXQJ12A8C134D94',
 'The Ballad Of Sleeping Beauty',
 'ARNF6401187FB57032',
 1994,
 305.162]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [122]:
from sql_queries import song_table_create, song_table_insert, song_table_drop

In [123]:
cur.execute(song_table_drop)

In [124]:
cur.execute(song_table_create)

In [125]:
cur.execute(song_table_insert, song_data)

~~Run `test.ipynb` to see if you've successfully added a record to this table.~~  
Use ```head_table()``` function instead

In [126]:
head_table(table="songs", conn=conn, cur=cur)

[('SONWXQJ12A8C134D94',
  'The Ballad Of Sleeping Beauty',
  'ARNF6401187FB57032',
  1994,
  305.162)]

#### Add all remaining song records

In [127]:
song_df = load_all_records(song_files, columns=["song_id", "title", "artist_id", "year", "duration"], 
                           to_disk=True, file_path=data_path, fname="song_data")

In [128]:
song_df.head()

,song_id,title,artist_id,year,duration
0,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,ARNF6401187FB57032,1994,305.16200
0,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,ARJIE2Y1187B994AB7,0,152.92036
0,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),ARPFHN61187FB575F6,0,279.97995
0,SOGDBUF12A8C140FAA,Intro,AR558FS1187FB45658,2003,75.67628
0,SONSKXP12A8C13A2C9,Native Soul,AR0IAWL1187B9A96D0,2003,197.19791


In [129]:
from sql_queries import song_queries

drop_and_load(query_list=song_queries, data=song_df, conn=conn, cur=cur)

In [130]:
#cur.execute(song_table_drop)
#cur.execute(song_table_create)
#bulk_insert(table="songs", file_path=data_path, fname="song_data", cur=cur)

In [131]:
head_table(table="songs", n=2, conn=conn, cur=cur)

[('SONWXQJ12A8C134D94',
  'The Ballad Of Sleeping Beauty',
  'ARNF6401187FB57032',
  1994,
  305.162),
 ('SOUPIRU12A6D4FA1E1',
  'Der Kleine Dompfaff',
  'ARJIE2Y1187B994AB7',
  0,
  152.92036)]

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [132]:
artist_data = first_df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values.tolist()
artist_data

['ARNF6401187FB57032',
 'Sophie B. Hawkins',
 'New York, NY [Manhattan]',
 40.79086,
 -73.96644]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [133]:
from sql_queries import artist_table_create, artist_table_insert, artist_table_drop

In [134]:
cur.execute(artist_table_drop)

In [135]:
cur.execute(artist_table_create)

In [136]:
cur.execute(artist_table_insert, artist_data)

~~Run `test.ipynb` to see if you've successfully added a record to this table.~~  
Use ```head_table()``` function instead

In [137]:
head_table(table="artists", conn=conn, cur=cur)

[('ARNF6401187FB57032',
  'Sophie B. Hawkins',
  'New York, NY [Manhattan]',
  40.79086,
  -73.96644)]

#### Add all remaining artist records

In [138]:
artists_df = load_all_records(song_files, 
                              columns=["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"],
                              to_disk=True, file_path=data_path, fname="artist_data")

In [139]:
artists_df.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARNF6401187FB57032,Sophie B. Hawkins,"New York, NY [Manhattan]",40.79086,-73.96644
0,ARJIE2Y1187B994AB7,Line Renaud,,NaN,NaN
0,ARPFHN61187FB575F6,Lupe Fiasco,"Chicago, IL",41.88415,-87.63241
0,AR558FS1187FB45658,40 Grit,,NaN,NaN
0,AR0IAWL1187B9A96D0,Danilo Perez,Panama,8.41770,-80.11278


In [140]:
from sql_queries import artist_queries
drop_and_load(query_list=artist_queries, data=artists_df, conn=conn, cur=cur)

In [141]:
#cur.execute(artist_table_drop)
#cur.execute(artist_table_create)
#bulk_insert(table="artists", file_path=data_path, fname="artist_data", cur=cur)

In [142]:
head_table(table="artists", conn=conn, cur=cur)

[('ARNF6401187FB57032',
  'Sophie B. Hawkins',
  'New York, NY [Manhattan]',
  40.79086,
  -73.96644),
 ('ARJIE2Y1187B994AB7', 'Line Renaud', '', nan, nan),
 ('ARPFHN61187FB575F6', 'Lupe Fiasco', 'Chicago, IL', 41.88415, -87.63241),
 ('AR558FS1187FB45658', '40 Grit', '', nan, nan),
 ('AR0IAWL1187B9A96D0', 'Danilo Perez', 'Panama', 8.4177, -80.11278)]

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [143]:
log_files = get_files(log_files)

In [144]:
log_files[:5]

['/home/workspace/data/log_data/2018/11/2018-11-09-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-27-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-02-events.json']

In [145]:
first_log = log_files[0]

In [146]:
!head $first_log -n 2

{"artist":"Muse","auth":"Logged In","firstName":"Harper","gender":"M","itemInSession":1,"lastName":"Barrett","length":209.50159,"level":"paid","location":"New York-Newark-Jersey City, NY-NJ-PA","method":"PUT","page":"NextSong","registration":1540685364796.0,"sessionId":275,"song":"Supermassive Black Hole (Twilight Soundtrack Version)","status":200,"ts":1541721977796,"userAgent":"\"Mozilla\/5.0 (Windows NT 6.3; WOW64) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.143 Safari\/537.36\"","userId":"42"}
{"artist":"Beastie Boys","auth":"Logged In","firstName":"Harper","gender":"M","itemInSession":2,"lastName":"Barrett","length":161.56689,"level":"paid","location":"New York-Newark-Jersey City, NY-NJ-PA","method":"PUT","page":"NextSong","registration":1540685364796.0,"sessionId":275,"song":"Lighten Up","status":200,"ts":1541722186796,"userAgent":"\"Mozilla\/5.0 (Windows NT 6.3; WOW64) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.143 Safari\/537.36\"","userId":"42"}


In [147]:
log_df = pd.read_json(log_files[0], orient="records", typ="frame", lines=True)

In [148]:
log_df.shape

(283, 18)

In [149]:
log_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [150]:
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [151]:
# convert ts column to proper datetime
log_df["ts"] = log_df["ts"].astype("datetime64[ms]")

In [152]:
# select only rows where page is 'NextSong'
log_df_filtered = log_df.loc[(log_df["page"] == "NextSong"), :]

In [153]:
time_dict = {
    "timestamp": log_df_filtered.ts,
    "hour": log_df_filtered.ts.dt.hour,
    "day": log_df_filtered.ts.dt.day,
    "week_of_year": log_df_filtered.ts.dt.weekofyear,
    "month": log_df_filtered.ts.dt.month,
    "year": log_df_filtered.ts.dt.year,
    "weekday": log_df_filtered.ts.dt.weekday
}

In [154]:
time_df = pd.DataFrame.from_dict(time_dict)

In [155]:
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,2018-11-09 00:06:17.796,0,9,45,11,2018,4
1,2018-11-09 00:09:46.796,0,9,45,11,2018,4
2,2018-11-09 00:12:27.796,0,9,45,11,2018,4
3,2018-11-09 00:14:52.796,0,9,45,11,2018,4
4,2018-11-09 00:17:44.796,0,9,45,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [156]:
from sql_queries import time_table_create, time_table_insert

In [157]:
cur.execute(time_table_create)

In [158]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

~~Run `test.ipynb` to see if you've successfully added records to this table.~~

In [159]:
head_table(table="time", conn=conn, cur=cur)

[(datetime.datetime(2018, 11, 9, 0, 6, 17, 796000), 0, 9, 45, 11, 2018, 4),
 (datetime.datetime(2018, 11, 9, 0, 9, 46, 796000), 0, 9, 45, 11, 2018, 4),
 (datetime.datetime(2018, 11, 9, 0, 12, 27, 796000), 0, 9, 45, 11, 2018, 4),
 (datetime.datetime(2018, 11, 9, 0, 14, 52, 796000), 0, 9, 45, 11, 2018, 4),
 (datetime.datetime(2018, 11, 9, 0, 17, 44, 796000), 0, 9, 45, 11, 2018, 4)]

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [160]:
user_df = log_df[["userId", "firstName", "lastName", "gender", "level"]]

In [161]:
user_df.head()

,userId,firstName,lastName,gender,level
0,42,Harper,Barrett,M,paid
1,42,Harper,Barrett,M,paid
2,42,Harper,Barrett,M,paid
3,42,Harper,Barrett,M,paid
4,42,Harper,Barrett,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [162]:
from sql_queries import user_table_create, user_table_insert, user_table_drop

In [163]:
cur.execute(user_table_drop)

In [164]:
cur.execute(user_table_create)

In [165]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

DataError: invalid input syntax for integer: ""
LINE 3: VALUES ('', NULL, NULL, NULL, 'paid')
                ^


~~Run `test.ipynb` to see if you've successfully added records to this table.~~

In [166]:
head_table(table="users", conn=conn, cur=cur)

[(42, 'Harper', 'Barrett', 'M', 'paid'),
 (49, 'Chloe', 'Cuevas', 'F', 'free'),
 (6, 'Cecilia', 'Owens', 'F', 'free'),
 (24, 'Layla', 'Griffin', 'F', 'paid'),
 (101, 'Jayden', 'Fox', 'M', 'free')]

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [170]:
from sql_queries import song_select, songplay_table_create, songplay_table_insert, song_table_drop

In [171]:
cur.execute(songplay_table_create)

In [172]:
for index, row in log_df_filtered.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [173]:
head_table("songplay", n=2, conn=conn, cur=cur)

[(1,
  datetime.datetime(2018, 11, 9, 0, 6, 17, 796000),
  42,
  'paid',
  None,
  None,
  275,
  'New York-Newark-Jersey City, NY-NJ-PA',
  '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'),
 (2,
  datetime.datetime(2018, 11, 9, 0, 9, 46, 796000),
  42,
  'paid',
  None,
  None,
  275,
  'New York-Newark-Jersey City, NY-NJ-PA',
  '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')]

# Close Connection to Sparkify Database

In [174]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [201]:
!python create_tables.py

In [202]:
!python etl.py

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file